PostProcessing Workbook

This notebook is for testing calibration and stacking functions in Obsy

First we define our object

In [54]:
import os
from astropy.io import fits
import logging
import sqlite3
import shutil
import uuid
from pathlib import Path
from datetime import datetime

class PostProcess(object):
    def __init__(self):
        print("PostProcess object created")
        self.DEBUG=True
        self.sourceFolder="/home/gtulloch/obsy/sample_data/Processing/data/"
        self.repoFolder="/home/gtulloch/obsy/sample_data/Processing/repo/"
        self.dbName=dbName = self.repoFolder+"obsy.db"
        # Set up Database
        self.con = sqlite3.connect(dbName)
        self.cur = self.con.cursor()
        self.createTables()

    def createTables(self):
        if self.DEBUG:
            self.cur.execute("DROP TABLE if exists fitsFile")
            self.cur.execute("DROP TABLE if exists fitsHeader")
        self.cur.execute("CREATE TABLE if not exists fitsFile(unid, date, filename)")
        self.cur.execute("CREATE TABLE if not exists fitsHeader(thisUNID, parentUNID, keyword, value)")
        return
    
    def submitFile(self,fileName, hdr):
        if "DATE-OBS" in hdr:
            uuidStr=uuid.uuid4()
            sqlStmt="INSERT INTO fitsFile (unid, date, filename) VALUES ('{0}','{1}','{2}')".format(uuidStr,hdr["DATE-OBS"],fileName)
            
            self.cur.execute(sqlStmt)
            self.con.commit()
                
            for card in hdr:
                if type(hdr[card]) not in [bool,int,float]:
                    keywordValue=str(hdr[card]).replace('\'',' ')
                else:
                    keywordValue = hdr[card]
                sqlStmt="INSERT INTO fitsHeader (thisUNID, parentUNID, keyword, value) VALUES ('{0}','{1}','{2}','{3}')".format(uuid.uuid4(),uuidStr,card,keywordValue)

                try:
                    cur.execute(sqlStmt)
                    con.commit()
                except sqlite3.Error as er:
                    exc_type, exc_value, exc_tb = sys.exc_info()

        else:
            print("Error: File not added to repo due to missing date is "+fileName)
            return 0
        
        return 1
    
    def registerFiles(self):
        # Scan the pictures folder
        print("Scanning "+self.sourceFolder)
        
        for root, dirs, files in os.walk(os.path.abspath(self.sourceFolder)):
            for file in files:
                file_name, file_extension = os.path.splitext(os.path.join(root, file))
                print("Processing "+os.path.join(root, file))
                # Ignore everything not a *fit* file
                if "fit" not in file_extension:
                    continue
                
                try:
                    hdul = fits.open(os.path.join(root, file))
                except ValueError as e:
                    print("Invalid FITS file. File not processed is "+str(os.path.join(root, file)))
                    continue   
                
                hdr = hdul[0].header
                if "FRAME" in hdr:
                    print(os.path.join(root, file))

                    # Create an os-friendly date
                    try:
                        datestr=hdr["DATE-OBS"].replace("T", " ")
                        datestr=datestr[0:datestr.find('.')]
                        dateobj=datetime.strptime(datestr, '%Y-%m-%d %H:%M:%S')
                        fitsDate=dateobj.strftime("%Y%m%d%H%M%S")
                    except ValueError as e:
                        print("Invalid date format in header. File not processed is "+str(os.path.join(root, file)))
                        continue

                    # Create a new standard name for the file based on what it is
                    if (hdr["FRAME"]=="Light"):
                        if ("OBJECT" in hdr):
                            newName=newName="{0}-{1}-{2}-{3}s-{4}x{5}-t{6}.fits".format(hdr["OBJECT"].replace(" ", ""),hdr["FILTER"],fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                        else:
                            print("Invalid object name in header. File not processed is "+str(os.path.join(root, file)))
                            continue
                    elif hdr["FRAME"]=="Dark" or hdr["FRAME"]=="Flat" or hdr["FRAME"]=="Bias":
                        newName="{0}-{1}-{2}s-{3}x{4}-t{5}.fits".format(hdr["FRAME"],fitsDate,hdr["EXPTIME"],hdr["XBINNING"],hdr["YBINNING"],hdr["CCD-TEMP"])
                    else:
                        print("File not processed as FRAME not recognized: "+str(os.path.join(root, file)))
                    hdul.close()

                    # Create the folder structure (if needed)
                    fitsDate=dateobj.strftime("%Y%m%d")
                    if (hdr["FRAME"]=="Light"):
                        newPath=self.repoFolder+"Light/{0}/{1}/".format(hdr["OBJECT"].replace(" ", ""),fitsDate)
                    elif hdr["FRAME"]=="Dark":
                        newPath=self.repoFolder+"Calibrate/{0}/{1}/{2}/".format(hdr["FRAME"],hdr["EXPTIME"],fitsDate)
                    elif hdr["FRAME"]=="Flat":
                        newPath=self.repoFolder+"Calibrate/{0}/{1}/{2}/".format(hdr["FRAME"],hdr["FILTER"],fitsDate)
                    elif hdr["FRAME"]=="Bias":
                        newPath=self.repoFolder+"Calibrate/{0}/{1}/".format(hdr["FRAME"],fitsDate)

                    if not os.path.isdir(newPath):
                        os.makedirs (newPath)

                    # If we can add the file to the database move it to the repo
                    if (submitFile(newPath+newName.replace(" ", ""),hdr)):
                        if self.DEBUG:
                            moveInfo="Moving {0} to {1}\n".format(os.path.join(root, file),newPath+newName)
                            print(moveInfo)
                        shutil.move(os.path.join(root, file),newPath+newName)
                    else:
                        print("Warning: File not added to repo is "+str(os.path.join(root, file)))
                else:
                    print("File not added to repo - no FRAME card - "+str(os.path.join(root, file)))   

Run the register program to populate the database. 

In [55]:
postProcessObj=PostProcess()
postProcessObj.registerFiles()

PostProcess object created
Scanning /home/gtulloch/obsy/sample_data/Processing/data/
Processing /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Lights.zip
Processing /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Bias.zip
Processing /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Flats.zip
Processing /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Dark.zip
Processing /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Dark/Dark/calibration frames_80sec_1x1_darks_frame19.fit
File not added to repo - no FRAME card - /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Dark/Dark/calibration frames_80sec_1x1_darks_frame19.fit
Processing /home/gtulloch/obsy/sample_data/Processing/data/DiegoColonnello-TrifidNebula-Dark/Dark/calibration frames_80sec_1x1_darks_frame20.fit
File not added to repo - no FRAME card - /home/gtulloch/obs

Lets see what was imported

In [56]:
postProcessObj.cur.execute("SELECT * FROM fitsFile")
files=postProcessObj.cur.fetchall()
print(files)

[]
